In [7]:
import sys 
sys.path.append('../../')  
from main import device_process_measurement_tomography, Quantities, Cross_Quantities
import QuantumTomography as qt 
import matplotlib.pyplot as plt 
import numpy as np 
from qiskit import QuantumCircuit, Aer, IBMQ 
from qiskit.compiler import transpile 
import json 
from qiskit.result import Result 
from qiskit.test.mock import FakeCasablanca  
from utils_qiskit import * 
from joblib import Parallel, delayed
import uuid



In [3]:
IBMQ.load_account()
device = FakeCasablanca()

ibmqfactory.load_account:WARNING:2022-10-14 16:11:29,340: Credentials are already in use. The existing account in the session will be replaced.


In [4]:
jobs = ['data/618b14fdeb64113dd7bb6080-output.json',
        'data/618b15007cdad6feb7ec066e-output.json',
        'data/618b150282271e07df2b280b-output.json',
        'data/618b15051452776ec0118c69-output.json']

data = []
for job in jobs:
    with open(job, 'r') as f:
        data.append( dict2results( json.load(f) ) )

results = CombineResults( data )
del data

In [5]:
dpmt = device_process_measurement_tomography( device)
circuits_dpmt = dpmt.circuits(label_qubits=False )

In [ ]:
def function():

    results_tomography = dpmt.fit( results, paralell=True, gate_set=True, resampling=2**13 )         
    results_single     = results_tomography.single                             
    results_double     = results_tomography.double                            
    results_gateset    = results_tomography.gateset   

    quantities = []
    pairs_qubits = []

    for idx1 in range(3):
        for idx2 in range(2):

            qubit0 = dpmt._parall_qubits[idx1][idx2][0]
            qubit1 = dpmt._parall_qubits[idx1][idx2][1]
            pairs_qubits.append( '({},{})'.format(qubit0,qubit1) ) 

            choi0=results_single.chois[qubit0]
            choi1=results_single.chois[qubit1]
            povm0=results_single.povms[qubit0]
            povm1=results_single.povms[qubit1]

            choi01=results_double[idx1].chois[idx2]
            povm01=results_double[idx1].povms[idx2]

            Fs0 = Quantities( povm0, choi0 )
            Fs1 = Quantities( povm1, choi1 )
            Fs01 = Quantities( povm01, choi01 )
            Cs  = Cross_Quantities( povm0, choi0, povm1, choi1, povm01, choi01 )

            quantities.append( Cs )
        
    return quantities

In [ ]:
results_mc = Parallel(n_jobs=-1)( delayed(function)() for _ in range(1000) )

In [ ]:
np.save( 'results_mc_'+str( uuid.uuid4() ), np.array( results_mc) )